In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('/Users/jackcook/Documents/Course notes/Data Sources/1.02.Multiple-linear-regression.csv')

## Inspect

In [3]:
df.head()

,SAT,"Rand 1,2,3",GPA
0,1714,1,2.40
1,1664,3,2.52
2,1760,3,2.54
3,1685,3,2.74
4,1693,2,2.83


In [4]:
df.describe()

,SAT,"Rand 1,2,3",GPA
count,84.000000,84.000000,84.000000
mean,1845.273810,2.059524,3.330238
std,104.530661,0.855192,0.271617
min,1634.000000,1.000000,2.400000
25%,1772.000000,1.000000,3.190000
50%,1846.000000,2.000000,3.380000
75%,1934.000000,3.000000,3.502500
max,2050.000000,3.000000,3.810000


# Regression - no scaling

In [5]:
x = df[['SAT','Rand 1,2,3']]
y = df.GPA

In [7]:
reg = LinearRegression()
reg.fit(x,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [9]:
reg.coef_, reg.intercept_

(array([ 0.00165354, -0.00826982]), 0.29603261264909486)

## Calculating R squared

In [14]:
reg.score(x,y)

0.4066811952814285

### Formula for Adjusted R Squared
n is number of observations, p is number of predictors

$$R^2{adj.} = 1 - (1-R^2)*\frac{n-1}{n-p-1}$$

In [17]:
def adjust_r_squared(x_array, rsq):
    n,p = x_array.shape
    adj_rsq = 1 - (1-rsq) * (n - 1)/(n - p - 1)
    return adj_rsq

In [18]:
adjust_r_squared(x,reg.score(x,y))

0.39203134825134023

### F regression (feature selection)

#### Caution, F regression doesn't take into account interrelation of the features!

In [19]:
from sklearn.feature_selection import f_regression

(array([56.04804786,  0.17558437]), array([7.19951844e-11, 6.76291372e-01]))

In [22]:
#F regression returns 2 values: F statistic and p value, we only want the p value
_, p_values = f_regression(x,y)
p_values

array([7.19951844e-11, 6.76291372e-01])

In [23]:
p_values.round(3)

array([0.   , 0.676])

SAT scores useful, Rand 1,2,3 is useless

## Summary table

In [25]:
reg_summary = pd.DataFrame(data=x.columns.values, columns=['Features'])
reg_summary

,Features
0,SAT
1,"Rand 1,2,3"


In [26]:
reg_summary['Coefficients'] = reg.coef_
reg_summary['p values'] = p_values.round(3)
reg_summary

,Features,Coefficients,p values
0,SAT,0.001654,0.000
1,"Rand 1,2,3",-0.008270,0.676


# Regression - with scaling

## Standardisation

Mean of 0, standard deviation of 1

In [27]:
from sklearn.preprocessing import StandardScaler

In [28]:
scaler = StandardScaler()

In [29]:
scaler.fit(x)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [30]:
x_scaled = scaler.transform(x) #Subtracts mean, divides by std

In [32]:
x_scaled[:5,:]

array([[-1.26338288, -1.24637147],
       [-1.74458431,  1.10632974],
       [-0.82067757,  1.10632974],
       [-1.54247971,  1.10632974],
       [-1.46548748, -0.07002087]])

## Regression

In [33]:
reg_scaled = LinearRegression()
reg_scaled.fit(x_scaled, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [38]:
reg_summary = pd.DataFrame(data=[['Intercept (Bias)'],['SAT'],['Rand 1,2,3']],
                           columns=['Features'])
reg_summary['Weights'] = reg_scaled.intercept_, reg_scaled.coef_[0], reg_scaled.coef_[1]

In [39]:
reg_summary

,Features,Weights
0,Intercept (Bias),3.330238
1,SAT,0.171814
2,"Rand 1,2,3",-0.007030


Weights is an ML term for coefficients

Bias is an ML term for the intercept

## Making predictions with the standardised coefficients (weights)

In [40]:
new_data = pd.DataFrame(data=[[1700,2],[1800,1]],columns=['SAT','Rand 1,2,3'])
new_data

,SAT,"Rand 1,2,3"
0,1700,2
1,1800,1


In [42]:
#This will look like nonsense as we haven't scaled the data!
reg_scaled.predict(new_data)

array([295.39979563, 312.58821497])

#### Re-use our standardisation scaler

In [43]:
new_data_scaled = scaler.transform(new_data)
new_data_scaled

array([[-1.39811928, -0.07002087],
       [-0.43571643, -1.24637147]])

In [44]:
reg_scaled.predict(new_data_scaled)

array([3.09051403, 3.26413803])